In [1]:
""" Module for writing artifical vcfs,
    for testing purposes """

' Module for writing artifical vcfs,\n    for testing purposes '

In [2]:
import pandas as pd
import os
import VCF
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [13]:
def write_vcf(outStr_, chrom, pos):
    """ routine for writing VCF files, from an existing dataframe. 
    essentially just adding in this horrible vcf header. """
    
    cwd = os.getcwd()
    
    with open(cwd + '/../cerebra/vcfheader.txt', 'r') as f:
        header = f.read()
        df = pd.DataFrame(columns=['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO'])
        
        chrom_ = 'chr' + str(chrom)
        pos_ = pos
        id_ = '.'
        ref_ = 'C'
        alt_ = 'A'
        qual_ = 60.28
        filter_ = '.'
        info_ = 'AC=2;AF=1.00;AN=2;DP=7;ExcessHet=3.0103;FS=0.000;MLEAC=1;MLEAF=0.500;MQ=3.00;QD=30.14;SOR=2.303'
        
        df = df.append({'CHROM': chrom_, 'POS': pos_, 'ID': id_, 'REF': ref_, 'ALT': alt_, 'QUAL': qual_, 'FILTER': filter_, 'INFO': info_}, ignore_index=True)
        
        output_VCF = outStr_
        with open(output_VCF, 'w') as vcf:
            vcf.write(header)

        df.to_csv(output_VCF, sep="\t", mode='a', index=False)

In [14]:
write_vcf('egfr_L858R.vcf', 7, 55191822)

In [15]:
VCF.dataframe('egfr_L858R.vcf')

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,chr7,55191822,.,C,A,60.28,.,AC=2;AF=1.00;AN=2;DP=7;ExcessHet=3.0103;FS=0.0...


In [9]:
VCF.dataframe('../cerebra/wrkdir/artificalVCF/E1_1000.vcf')

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,chr7,55191821,.,C,T,60.28,.,AC=2;AF=1.00;AN=2;DP=7;ExcessHet=3.0103;FS=0.0...


In [10]:
VCF.dataframe('../cerebra/wrkdir/artificalVCF/E2_1000.vcf')

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,7,55191822,.,C,A,60.28,.,AC=2;AF=1.00;AN=2;DP=7;ExcessHet=3.0103;FS=0.0...
